see: [Quickstart Tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

from smot.common.runtime import build_paths, notebooks

In [14]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root=build_paths.data_root(),
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root=build_paths.data_root(),
    train=False,
    download=True,
    transform=ToTensor(),
)

In [15]:
batch_size = 128

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([128, 1, 28, 28])
Shape of y:  torch.Size([128]) torch.int64


In [16]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [18]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [19]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [20]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298176  [    0/60000]
loss: 2.273001  [12800/60000]
loss: 2.264601  [25600/60000]
loss: 2.258830  [38400/60000]
loss: 2.238247  [51200/60000]
Test Error: 
 Accuracy: 30.5%, Avg loss: 2.228121 

Epoch 2
-------------------------------
loss: 2.228462  [    0/60000]
loss: 2.193598  [12800/60000]
loss: 2.188980  [25600/60000]
loss: 2.186174  [38400/60000]
loss: 2.153789  [51200/60000]
Test Error: 
 Accuracy: 41.6%, Avg loss: 2.142264 

Epoch 3
-------------------------------
loss: 2.143719  [    0/60000]
loss: 2.093665  [12800/60000]
loss: 2.087827  [25600/60000]
loss: 2.081807  [38400/60000]
loss: 2.037716  [51200/60000]
Test Error: 
 Accuracy: 49.7%, Avg loss: 2.019400 

Epoch 4
-------------------------------
loss: 2.018414  [    0/60000]
loss: 1.950547  [12800/60000]
loss: 1.941099  [25600/60000]
loss: 1.926520  [38400/60000]
loss: 1.878854  [51200/60000]
Test Error: 
 Accuracy: 53.3%, Avg loss: 1.850329 

Epoch 5
------------------------

In [21]:
model_path = notebooks.output_path("model.pth")
model_path

'/home/crutcher/git/smot/build/smot/scratch/pytorch/basics/model.pth'

In [22]:
torch.save(model.state_dict(), model_path)
print(f"Saved PyTorch Model State to {model_path}")

Saved PyTorch Model State to /home/crutcher/git/smot/build/smot/scratch/pytorch/basics/model.pth


In [23]:
model = NeuralNetwork()
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [24]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
